In [1]:
import os

In [2]:
%pwd

'c:\\Users\\SaiVenkataSriramChow\\Downloads\\chest-cancer-classification\\research'

In [3]:
os.chdir('c:\\Users\\SaiVenkataSriramChow\\Downloads\\chest-cancer-classification')
%pwd

'c:\\Users\\SaiVenkataSriramChow\\Downloads\\chest-cancer-classification'

In [4]:
import dagshub
dagshub.init(repo_owner='sriram-2490', repo_name='my-first-repo', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as sriram-2490

Initialized MLflow to track repo "sriram-2490/my-first-repo"

Repository sriram-2490/my-first-repo initialized!

In [5]:

import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [8]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config


In [10]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [11]:
import mlflow
import dagshub
import tensorflow as tf
from pathlib import Path
from urllib.parse import urlparse

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)  # Use self.model instead of model
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    def log_into_mlflow(self):
        # Initialize Dagshub and MLflow
        dagshub.init(repo_owner='sriram-2490', repo_name='my-first-repo', mlflow=True)

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
            if tracking_url_type_store != "file":
                # Register the model
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [12]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-03-25 00:12:49,582: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-25 00:12:49,585: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-25 00:12:49,586: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 6s 861ms/step - loss: 27.6326 - accuracy: 0.5686
[2025-03-25 00:12:56,325: INFO: common: json file saved at: scores.json]
[2025-03-25 00:12:56,608: INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/sriram-2490/my-first-repo "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "sriram-2490/my-first-repo"

[2025-03-25 00:12:56,608: INFO: helpers: Initialized MLflow to track repo "sriram-2490/my-first-repo"]


Repository sriram-2490/my-first-repo initialized!

[2025-03-25 00:12:56,624: INFO: helpers: Repository sriram-2490/my-first-repo initialized!]


2025/03/25 00:12:57 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2025-03-25 00:12:58,059: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\SAIVEN~1\AppData\Local\Temp\tmpwyo410j7\model\data\model\assets
[2025-03-25 00:12:58,979: INFO: builder_impl: Assets written to: C:\Users\SAIVEN~1\AppData\Local\Temp\tmpwyo410j7\model\data\model\assets]


c:\Users\SaiVenkataSriramChow\anaconda3\envs\cancer\lib\site-packages\_distutils_hack\__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2025/03/25 00:13:38 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 5
Created version '5' of model 'VGG16Model'.
